<a href="https://colab.research.google.com/github/BrunoFoggiatto/Pibic-Visao_Computacional/blob/Modelos_Treinamento/Treinar_Modelo_Yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_yaml = """path: /content/drive/MyDrive/Colab Notebooks/Dataset_250

train: /content/drive/MyDrive/Colab Notebooks/Dataset_250/train
val: /content/drive/MyDrive/Colab Notebooks/Dataset_250/val

nc: 37
classes: ['Placa', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
"""

dataset_yaml_path = "/content/drive/MyDrive/Colab Notebooks/Dataset_250/dataset.yaml"

with open(dataset_yaml_path, "w", encoding="utf-8") as file:
    file.write(dataset_yaml)

import yaml
try:
    with open(dataset_yaml_path, "r", encoding="utf-8") as file:
        data = yaml.safe_load(file)
    print(data)
except yaml.YAMLError as e:
    print("\nErro ao carregar o YAML:", e)

{'path': '/content/drive/MyDrive/Colab Notebooks/Dataset_250', 'train': '/content/drive/MyDrive/Colab Notebooks/Dataset_250/train', 'val': '/content/drive/MyDrive/Colab Notebooks/Dataset_250/val', 'nc': 37, 'classes': ['Placa', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']}


In [4]:
!pip install easyocr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 19.0 MB/s eta 0:00:00


In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.1/994.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 820.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [ ]:
import os
import shutil
import random

orig_train_dir = "/content/drive/MyDrive/Colab Notebooks/Dataset/train"
orig_val_dir = "/content/drive/MyDrive/Colab Notebooks/Dataset/val"
orig_test_dir = "/content/drive/MyDrive/Colab Notebooks/Dataset/test"

new_dataset_dir = "/content/drive/MyDrive/Colab Notebooks/Dataset_250"
new_train_dir = os.path.join(new_dataset_dir, "train")
new_val_dir = os.path.join(new_dataset_dir, "val")

for dir in [new_train_dir, new_val_dir]:
    os.makedirs(dir, exist_ok=True)

def copy_fixed_number(source_dir, target_dir, num_files):
    files = [f for f in os.listdir(source_dir) if f.endswith(".jpg") or f.endswith(".png")]
    files = sorted(files)
    num_files = min(num_files, len(files))
    sampled_files = random.sample(files, num_files)

    for file in sampled_files:
        base_name, ext = os.path.splitext(file)
        image_path = os.path.join(source_dir, file)
        text_path = os.path.join(source_dir, base_name + ".txt")

        shutil.copy(image_path, os.path.join(target_dir, file))
        if os.path.exists(text_path):
            shutil.copy(text_path, os.path.join(target_dir, base_name + ".txt"))

copy_fixed_number(orig_train_dir, new_train_dir, 400)
copy_fixed_number(orig_val_dir, new_val_dir, 400)

train_size = len([f for f in os.listdir(new_train_dir) if f.endswith(".jpg") or f.endswith(".png")])
val_size = len([f for f in os.listdir(new_val_dir) if f.endswith(".jpg") or f.endswith(".png")])


In [ ]:
from ultralytics import YOLO

model_path = "/content/drive/MyDrive/Colab Notebooks/Yolo/yolo11n.pt"

model = YOLO(model_path)

model.train(
    data="/content/drive/MyDrive/Colab Notebooks/Dataset_250/dataset.yaml",
    epochs=1,
    patience=10,
    imgsz=640,
    batch=16,
    project="/content/drive/MyDrive/Colab Notebooks/Yolo",
    name="treinamento_yolo11n",  #
)

print("Treinamento finalizado!")


Ultralytics 8.3.93 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/Colab Notebooks/Yolo/yolo11n.pt, data=/content/drive/MyDrive/Colab Notebooks/Dataset_250/dataset.yaml, epochs=1, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Colab Notebooks/Yolo, name=treinamento_yolo11n6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=N

100%|██████████| 755k/755k [00:00<00:00, 16.8MB/s]


Overriding model.yaml nc=80 with nc=37

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /content/drive/MyDrive/Colab Notebooks/Dataset_250/train.cache... 870 images, 0 backgrounds, 0 corrupt: 100%|██████████| 870/870 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

Plotting labels to /content/drive/MyDrive/Colab Notebooks/Yolo/treinamento_yolo11n6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000244, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /content/drive/MyDrive/Colab Notebooks/Yolo/treinamento_yolo11n6
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G     0.8922      5.213     0.9769         10        640: 100%|██████████| 55/55 [16:17<00:00, 17.77s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [14:19<00:00, 31.84s/it]


                   all        840        840          0          0          0          0

1 epochs completed in 0.515 hours.
Optimizer stripped from /content/drive/MyDrive/Colab Notebooks/Yolo/treinamento_yolo11n6/weights/last.pt, 5.5MB
Optimizer stripped from /content/drive/MyDrive/Colab Notebooks/Yolo/treinamento_yolo11n6/weights/best.pt, 5.5MB

Validating /content/drive/MyDrive/Colab Notebooks/Yolo/treinamento_yolo11n6/weights/best.pt...
Ultralytics 8.3.93 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLO11n summary (fused): 100 layers, 2,589,367 parameters, 0 gradients, 6.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [02:25<00:00,  5.37s/it]


                   all        840        840          0          0          0          0
               class_0        840        840          0          0          0          0
Speed: 1.4ms preprocess, 148.3ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to /content/drive/MyDrive/Colab Notebooks/Yolo/treinamento_yolo11n6
Treinamento finalizado!
